# MSBD 5001 Indivisual Project
`Author: YUAN Yan Zhe`

First, i extract features like year,quartar,date,etc. from the column called `date` in both train and test set, then put the features into the regressor, i choose xgboost as the regressor and use gridsearch as the fine-tuning method.


There are two ideas after extracting features:
- use the features directly in the xgboost regressor. 
- use `pd.get_dummy()` to in the format of one-hot, then put the dummied features into the xgbregressor.
- after experiments, the result of the first method is better than the second one, which means there is no need to dummy featues.


Due to the upload limit, other experiments done with features dummied are done in the account:`ZZZ7779`. However, results come from this ipynb is relatively better, so i use this one as the final .ipynb file.


## Feature Engineering 

In [26]:
# import packages

!pip install datetime
import pandas as pd
import numpy as np
import re
from datetime import datetime
#import datetime
import time


In [27]:
# load data

train_data = pd.read_csv('Data/train.csv')
test_data = pd.read_csv('Data/test.csv')
submission = pd.read_csv('Data/sampleSubmission.csv')

### train data

In [28]:
train_data.head()

,id,date,speed
0,0,1/1/2017 0:00,43.002930
1,1,1/1/2017 1:00,46.118696
2,2,1/1/2017 2:00,44.294158
3,3,1/1/2017 3:00,41.067468
4,4,1/1/2017 4:00,46.448653


In [29]:
# transform date format

train_data['date']=pd.to_datetime(train_data['date'],format='%d/%m/%Y %H:%M')
train_data

,id,date,speed
0,0,2017-01-01 00:00:00,43.002930
1,1,2017-01-01 01:00:00,46.118696
2,2,2017-01-01 02:00:00,44.294158
3,3,2017-01-01 03:00:00,41.067468
4,4,2017-01-01 04:00:00,46.448653
...,...,...,...
14001,14001,2018-12-31 12:00:00,19.865269
14002,14002,2018-12-31 15:00:00,17.820375
14003,14003,2018-12-31 16:00:00,12.501851
14004,14004,2018-12-31 18:00:00,15.979319


In [30]:
# get features for date data of training set

train_data['year'] = train_data['date'].dt.year
train_data['quarter'] = train_data['date'].dt.quarter
train_data['month'] = train_data['date'].dt.month
train_data['week_of_year'] = train_data['date'].dt.weekofyear
train_data['day'] = train_data['date'].dt.day
train_data['day_of_week'] = train_data['date'].dt.dayofweek
train_data['day_of_year'] = train_data['date'].dt.dayofyear
train_data['hour'] = train_data['date'].dt.hour
train_data['is_traffic_time'] = train_data['hour'].apply(lambda hour: 1 if hour in [9,10,18,19] else 0)

# magic feature: check if the date is a holiday in HK
def is_holiday_date(date):
    timestamp = re.split(' ', str(datetime.strptime(re.split(' ', str(date))[0],"%Y-%m-%d")))[0]
    if timestamp in holiday_date:
        return 1
    return 0
holiday_date = ['2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14','2017-04-15','2017-04-17','2017-05-01','2017-05-03','2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26','2018-01-01','2018-02-16','2018-02-17','2018-02-18','2018-03-30','2018-03-31','2018-04-02','5/4/2018','2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17','2018-12-25','2018-12-26']
train_data['holiday'] = train_data["date"].apply(lambda date: is_holiday_date(date))


/Users/software/anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [31]:
train_data.dtypes

id                          int64
date               datetime64[ns]
speed                     float64
year                        int64
quarter                     int64
month                       int64
week_of_year                int64
day                         int64
day_of_week                 int64
day_of_year                 int64
hour                        int64
is_traffic_time             int64
holiday                     int64
dtype: object

In [32]:
# Feature Expansion
# get features in one-hot format, adding the dimensions which is goood for regressors to learn. 
#dummy_feature = pd.get_dummies(train_data, columns=['hour','day_of_week','week_of_year','month','quarter','day','year'])

# Select featues
train_data = train_data[['speed','day_of_year','is_traffic_time','holiday','hour','day_of_week','week_of_year','month','quarter','day','year']]


In [33]:
train_data

,speed,day_of_year,is_traffic_time,holiday,hour,day_of_week,week_of_year,month,quarter,day,year
0,43.002930,1,0,0,0,6,52,1,1,1,2017
1,46.118696,1,0,0,1,6,52,1,1,1,2017
2,44.294158,1,0,0,2,6,52,1,1,1,2017
3,41.067468,1,0,0,3,6,52,1,1,1,2017
4,46.448653,1,0,0,4,6,52,1,1,1,2017
...,...,...,...,...,...,...,...,...,...,...,...
14001,19.865269,365,0,0,12,0,1,12,4,31,2018
14002,17.820375,365,0,0,15,0,1,12,4,31,2018
14003,12.501851,365,0,0,16,0,1,12,4,31,2018
14004,15.979319,365,1,0,18,0,1,12,4,31,2018


In [34]:
# Split features and label into train_x and train_y

train_y = train_data['speed']
train_x = train_data.drop('speed',axis=1)
print(train_x.columns)
print(len(train_y))

Index(['day_of_year', 'is_traffic_time', 'holiday', 'hour', 'day_of_week',
       'week_of_year', 'month', 'quarter', 'day', 'year'],
      dtype='object')
14006


In [35]:
train_y

0        43.002930
1        46.118696
2        44.294158
3        41.067468
4        46.448653
           ...    
14001    19.865269
14002    17.820375
14003    12.501851
14004    15.979319
14005    40.594183
Name: speed, Length: 14006, dtype: float64

In [36]:
print(len(train_x.columns))

10


### test data
do the same thing as train data

In [37]:
test_data

,id,date
0,0,1/1/2018 2:00
1,1,1/1/2018 5:00
2,2,1/1/2018 7:00
3,3,1/1/2018 8:00
4,4,1/1/2018 10:00
...,...,...
3499,3499,31/12/2018 17:00
3500,3500,31/12/2018 19:00
3501,3501,31/12/2018 21:00
3502,3502,31/12/2018 22:00


In [38]:
test_data['date']=pd.to_datetime(test_data['date'],format='%d/%m/%Y %H:%M')
test_data

,id,date
0,0,2018-01-01 02:00:00
1,1,2018-01-01 05:00:00
2,2,2018-01-01 07:00:00
3,3,2018-01-01 08:00:00
4,4,2018-01-01 10:00:00
...,...,...
3499,3499,2018-12-31 17:00:00
3500,3500,2018-12-31 19:00:00
3501,3501,2018-12-31 21:00:00
3502,3502,2018-12-31 22:00:00


In [39]:
# get features for test data

test_data['year'] = test_data['date'].dt.year
test_data['quarter'] = test_data['date'].dt.quarter
test_data['month'] = test_data['date'].dt.month
test_data['week_of_year'] = test_data['date'].dt.weekofyear
test_data['day'] = test_data['date'].dt.day
test_data['day_of_week'] = test_data['date'].dt.dayofweek
test_data['day_of_year'] = test_data['date'].dt.dayofyear
test_data['hour'] = test_data['date'].dt.hour
test_data['is_traffic_time'] = test_data['hour'].apply(lambda hour: 1 if hour in [9,10,18,19] else 0)


def is_holiday_date(date):
    timestamp = re.split(' ', str(datetime.strptime(re.split(' ', str(date))[0],"%Y-%m-%d")))[0]
    if timestamp in holiday_date:
        return 1
    return 0
holiday_date = ['2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14','2017-04-15','2017-04-17','2017-05-01','2017-05-03','2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26','2018-01-01','2018-02-16','2018-02-17','2018-02-18','2018-03-30','2018-03-31','2018-04-02','5/4/2018','2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17','2018-12-25','2018-12-26']
test_data['holiday'] = test_data["date"].apply(lambda date: is_holiday_date(date))


/Users/software/anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [40]:
#dummy_feature_test =
test_x = test_data[['day_of_year','is_traffic_time','holiday','hour','day_of_week','week_of_year','month','quarter','day','year']]
# test_y = predict_y
print(test_x.columns)
test_x

Index(['day_of_year', 'is_traffic_time', 'holiday', 'hour', 'day_of_week',
       'week_of_year', 'month', 'quarter', 'day', 'year'],
      dtype='object')


,day_of_year,is_traffic_time,holiday,hour,day_of_week,week_of_year,month,quarter,day,year
0,1,0,1,2,0,1,1,1,1,2018
1,1,0,1,5,0,1,1,1,1,2018
2,1,0,1,7,0,1,1,1,1,2018
3,1,0,1,8,0,1,1,1,1,2018
4,1,1,1,10,0,1,1,1,1,2018
...,...,...,...,...,...,...,...,...,...,...
3499,365,0,0,17,0,1,12,4,31,2018
3500,365,1,0,19,0,1,12,4,31,2018
3501,365,0,0,21,0,1,12,4,31,2018
3502,365,0,0,22,0,1,12,4,31,2018


## XGBoost modeling

- After many times of fine-tuning(gredsearch) process, the parameters adjustment range resuces to: 

In [52]:
# fine-tuning on parameters of xgb

from sklearn.model_selection import GridSearchCV
import xgboost as xgb


#cv_params = {'learning_rate': [0.02,0.04,0.06,0.08],
#             'n_estimators': [600, 700, 800],
#             'subsample': [0.8,0.9], 
#             'max_depth': [4,6,8],
#             'colsample_bytree': [0.8,0.85,0.9], # 0.75 
#             'min_child_weight': [6,7,8],
#            }

cv_params = {'learning_rate': [0.035,0.04,0.045],
             'n_estimators': [740,760,780,800],
            }
other_params = {'max_depth': 8,'min_child_weight': 6,'subsample': 0.9,'colsample_bytree': 0.8,'seed': 0,'gamma': 0.8,'reg_alpha': 13,'reg_lambda': 5}

best_model = GridSearchCV(estimator=xgb.XGBRegressor(**other_params), param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=-1)
best_model.fit(train_x, train_y)
result = best_model.cv_results_
print('Averaged Score:{}'.format(result['mean_test_score']))
print('Corresponding Parameters:{}'.format(result['params']))
print('Best Para:{}'.format(best_model.best_params_))
print('Best Score:{}'.format(best_model.best_score_))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   35.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.0min finished


Averaged Score:[0.90870598 0.9085262  0.90839255 0.9082574  0.90749668 0.90732723
 0.90720385 0.9070302  0.90671606 0.90656176 0.90649615 0.90635493]
Corresponding Parameters:[{'learning_rate': 0.035, 'n_estimators': 740}, {'learning_rate': 0.035, 'n_estimators': 760}, {'learning_rate': 0.035, 'n_estimators': 780}, {'learning_rate': 0.035, 'n_estimators': 800}, {'learning_rate': 0.04, 'n_estimators': 740}, {'learning_rate': 0.04, 'n_estimators': 760}, {'learning_rate': 0.04, 'n_estimators': 780}, {'learning_rate': 0.04, 'n_estimators': 800}, {'learning_rate': 0.045, 'n_estimators': 740}, {'learning_rate': 0.045, 'n_estimators': 760}, {'learning_rate': 0.045, 'n_estimators': 780}, {'learning_rate': 0.045, 'n_estimators': 800}]
Best Para:{'learning_rate': 0.035, 'n_estimators': 740}
Best Score:0.9087059809749682


In [53]:
# Make predictions

prediction = best_model.predict(test_x)

In [54]:
# Submission

submission['speed'] = pd.DataFrame(prediction)
submission

,id,speed
0,0,48.117313
1,1,48.059593
2,2,36.608330
3,3,30.368513
4,4,39.407135
...,...,...
3499,3499,12.686603
3500,3500,27.390425
3501,3501,46.174721
3502,3502,40.826736


In [55]:
submission.to_csv("xgboost_12.csv", index=False)